In [29]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import re
import yfinance as yf

In [30]:
df = pd.read_csv('Assets\CONVICTIONSUMMARY_270_opt3_best2(1).csv', sep='|')

In [31]:
df2 = pd.DataFrame()
df2 = df.loc[df['TYPE'] == 'MERGED']

Extract Time and Date

In [32]:
df2["Time"] = df2.apply(lambda row: re.split(" ",row.DATE)[0], axis=1)

df2["DATE"] = df2.apply(lambda row: re.split(" ",row.DATE)[-1],axis=1)

<ipython-input-32-0de3e1e305d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Time"] = df2.apply(lambda row: re.split(" ",row.DATE)[0], axis=1)
<ipython-input-32-0de3e1e305d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["DATE"] = df2.apply(lambda row: re.split(" ",row.DATE)[-1],axis=1)


In [33]:
df2.dtypes

DATE             object
ID               object
STOCK            object
SECTOR           object
TYPE             object
CONVICTIONS     float64
Time             object
dtype: object

In [34]:
df2['DATE'] = pd.to_datetime(
                          df2['DATE'],
                          format='%Y-%m-%d')

<ipython-input-34-3660468bee21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['DATE'] = pd.to_datetime(


In [35]:
df2 = df2.sort_values(by="DATE")

In [36]:
df2.dtypes

DATE            datetime64[ns]
ID                      object
STOCK                   object
SECTOR                  object
TYPE                    object
CONVICTIONS            float64
Time                    object
dtype: object

## Cleaning

In [37]:
df2.columns = ['DATE', 'ID', 'STOCK',"SECTOR","TYPE","SCORE","TIME"]

In [38]:
df2 = df2.loc[df2['salary'] != 0]

KeyError: 'salary'

## Adding returns

### gettting info

In [ ]:
list_of_companies = df2["STOCK"].unique()
list_of_dates = df2["DATE"].unique()

In [ ]:
start_date = df2.DATE.min()
end_date = df2.DATE.max()

In [ ]:
final_date = end_date + pd.DateOffset(months=12)
print(final_date)

2005-07-14 00:00:00


### Define functions

In [39]:
def get_history_data(ticker_symbol, start_date, end_date):
    tickerData = yf.Ticker(ticker_symbol)
    tickerDf = tickerData.history(period='1d', start=start_date, end=end_date)
    tickerDf["STOCK"] = ticker_symbol

    return tickerDf

    

In [40]:
def get_return_on_date(date,df):
  try:
    x = df.loc[date]
    x = float(x['Close'])
  except:
    x = 0.0
  return x

In [82]:
def calculate_returns(start_date, df):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MRerturn','12MReturn'])
    
    date1m = start_date + pd.DateOffset(months=1)
    date3m = start_date + pd.DateOffset(months=3)
    date6m = start_date + pd.DateOffset(months=6)
    date12m = start_date +pd.DateOffset(months=12)

    name = df['STOCK']
    try:
        current = df.loc[start_date]
    except:
        return returns
    data1m = get_return_on_date(date1m,df)
    data3m = get_return_on_date(date3m,df)
    data6m = get_return_on_date(date6m,df)
    data12m = get_return_on_date(date12m,df)

    current = float(current['Close'])

    return1m = (data1m - current)/current
    return3m = (data3m - current)/current
    return6m = (data6m - current)/current
    return12m = (data6m - current)/current


    returns = pd.DataFrame([[start_date,name[0],return1m, return3m, return6m,return12m]], columns=['DATE','STOCK','1MReturn',"3MReturn","6MReturn","12MReturn"])

 

    return returns

In [42]:
def calculate_returns_for_date(list_of_dates,df):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MReturn','12MReturn'])
    for d in list_of_dates:
        temp = calculate_returns(pd.to_datetime(d),df)
        
        returns = returns.append(temp, ignore_index=True)
    
    return returns

In [55]:
def calculate_returns_for_companies(list_of_dates,list_of_companies,start_date, end_date):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MReturn','12MReturn'])
    for c in list_of_companies:
        data = get_history_data(c,start_date, end_date)
        len_of_data = data.shape[0]
        if len_of_data !=0:
            returns_for_company = calculate_returns_for_date(list_of_dates,data)
            returns = returns.append(returns_for_company, ignore_index=True)
        
    return returns
    

### Get Data

In [92]:
test = calculate_returns_for_companies(list_of_dates, list_of_companies,start_date, end_date)

- PKZ: No data found for this date range, symbol may be delisted
- PALDF: No data found, symbol may be delisted
- CHRS.XX1: No data found, symbol may be delisted
- AGU: No data found for this date range, symbol may be delisted
- BJS: No data found for this date range, symbol may be delisted
- AAN.XX1: No data found, symbol may be delisted
- NMG.B: No data found for this date range, symbol may be delisted
- HBG: Data doesn't exist for startDate = 1076454000, endDate = 1089756000
- SSINQ: No data found, symbol may be delisted
- DSCP: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- EPL: No data found for this date range, symbol may be delisted
- APCC.XX9: No data found, symbol may be delisted
- PP.XX9: No data found, symbol may be delisted
- INET.XX11: No data found, symbol may be delisted
- SYD: Data doesn't exist for startDate = 1076454000, endDate = 1089756000
- SKO: Data doesn't exist for startDate = 1076454000, endDate = 1089

In [94]:
test.tail(20)

,DATE,STOCK,1MReturn,3MReturn,6MReturn,12MReturn,6MRerturn
12715,2004-02-25,GBX,0.097403,0.025974,-1.0,-1.0,NaN
12716,2004-03-03,GBX,-1.000000,0.097834,-1.0,-1.0,NaN
12717,2004-03-10,GBX,-1.000000,0.098773,-1.0,-1.0,NaN
12718,2004-03-17,GBX,-1.000000,0.118218,-1.0,-1.0,NaN
12719,2004-03-24,GBX,-1.000000,0.076663,-1.0,-1.0,NaN
12720,2004-03-31,GBX,0.000593,0.127887,-1.0,-1.0,NaN
12721,2004-04-07,GBX,-0.001274,0.251592,-1.0,-1.0,NaN
12722,2004-04-14,GBX,-0.092592,-1.000000,-1.0,-1.0,NaN
12723,2004-04-21,GBX,-0.112426,-1.000000,-1.0,-1.0,NaN
12724,2004-04-28,GBX,0.001744,-1.000000,-1.0,-1.0,NaN
